In [ ]:
import socket
import struct
import time
import math

import socket
import struct
import math
import time

class QuicServer:
    local_port = 12345  # Constant for local port
    packet_size = 1024  # Constant for packet size in bytes

    def __init__(self):
        """
        Initialize the QuicServer class.
        """
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind(('', self.local_port))  # Bind to any available interface
        self.stream_data = {}  # Dictionary to store data for each stream
        self.start_time = {}  # Dictionary to store start time for each stream
        self.total_bytes_received = 0  # Total bytes received across all streams
        self.total_packets_received = 0  # Total packets received across all streams

    def process_initial_message(self):
        """
        Receive the initial message and extract the sender's IP, number of streams, and file size.
        
        Returns:
            sender_ip (str): The sender's IP address.
            num_streams (int): The number of streams.
            file_size (int): The size of the file.
        """
        initial_message, sender_address = self.socket.recvfrom(1024)
        sender_ip, _ = sender_address
        num_streams, file_size = struct.unpack("!II", initial_message)
        return sender_ip, num_streams, file_size

    def update_statistics(self, stream_id, data):
        """
        Process incoming data on the stream.
        
        Args:
            stream_id (int): The ID of the stream.
            data (bytes): The data received on the stream.
        """
        # Track the number of bytes and packets received for each stream
        if stream_id not in self.stream_data:
            self.stream_data[stream_id] = {"bytes_received": 0, "packets_received": 0}
            self.start_time[stream_id] = time.time()
        self.stream_data[stream_id]["bytes_received"] += len(data)  # Update bytes received
        self.stream_data[stream_id]["packets_received"] += 1  # Update packets received

        # Update total bytes and packets received across all streams
        self.total_bytes_received += len(data)  # Update total bytes received
        self.total_packets_received += 1  # Update total packets received

    def receive_packets(self, stream_id, file_size):
        """Receive and assemble chunks of data for each stream."""
        received_data = b""
        num_packets = math.ceil(file_size / self.packet_size)  # Calculate number of packets
        remaining_bytes = file_size
        
        for _ in range(num_packets):
            data, _ = self.socket.recvfrom(self.packet_size)  # Receive packet of size packet_size
            received_data += data
            self.update_statistics(stream_id, data)  # Process data after each packet received
            remaining_bytes -= len(data)  # Update remaining bytes
        
        return received_data

    def calculate_metrics(self):
        """Calculate metrics for each stream and total metrics."""
        metrics = {}
        current_time = time.time()
        total_elapsed_time = current_time - self.start_time[1]  # Use stream 1 as reference
        for stream_id, data in self.stream_data.items():
            elapsed_time = current_time - self.start_time[stream_id]
            metrics[stream_id] = {
                "bytes_transferred": data["bytes_received"],
                "packets_transferred": data["packets_received"],
                "average_bytes_per_second": data["bytes_received"] / elapsed_time,
                "average_packets_per_second": data["packets_received"] / elapsed_time,
            }
        total_metrics = {
            "total_bytes_per_second": self.total_bytes_received / total_elapsed_time,
            "total_packets_per_second": self.total_packets_received / total_elapsed_time
        }
        return metrics, total_metrics

    def print_statistics(self, metrics, total_metrics):
        """Print statistics for each stream and total metrics."""
        for stream_id, metric in metrics.items():
            print(f"Stream {stream_id}:")
            print(f"Bytes transferred: {metric['bytes_transferred']}")
            print(f"Packets transferred: {metric['packets_transferred']}")
            print(f"Average bytes per second: {metric['average_bytes_per_second']}")
            print(f"Average packets per second: {metric['average_packets_per_second']}")
        print("\nTotal Metrics:")
        print(f"Total bytes per second: {total_metrics['total_bytes_per_second']}")
        print(f"Total packets per second: {total_metrics['total_packets_per_second']}")

# Create QuicServer instance
server = QuicServer()

# Receive the initial message and extract the sender's IP, number of streams, and file size
sender_ip, num_streams, file_size = server.process_initial_message()

# Process data for each stream and assemble the file
for stream_id in range(1, num_streams + 1):
    server.update_statistics(stream_id, b"")  # Initialize data for the stream
    received_data = server.receive_packets(stream_id, file_size)

# Calculate metrics for each stream and total metrics
metrics, total_metrics = server.calculate_metrics()

# Print statistics for each stream and total metrics
server.print_statistics(metrics, total_metrics)